# Testing the API

In [ ]:
import requests
port = 5000
# Note: Sometimes I use port 5001 locally, but flask is typically 5000 ^ set your port with the variable above

###### To Use:
# docker pull jthet/hurricane-prediction:latest
# docker run -p 5000:5000 -it --rm jthet/hurricane-prediction:latest

# Inference server must be running to execute the requests.

### Route: /help

In [5]:
'''
Provides an overview and usage examples for the available API endpoints.

Example: curl http://127.0.0.1:5000/help
'''
url = f'http://127.0.0.1:{port}/help'
response = requests.get(url)
print(response.text)

{
  "/model/change": {
    "description": "Changes the TensorFlow model used by the server.",
    "example": "curl -X POST -H 'Content-Type: application/json' -d '{\"model_name\": \"vgg\"}' http://127.0.0.1:5001/model/change",
    "method": "POST"
  },
  "/model/info": {
    "description": "Provides basic information about the currently loaded TensorFlow model.",
    "example": "curl http://127.0.0.1:5001/model/info",
    "method": "GET"
  },
  "/model/models": {
    "description": "Lists the available TensorFlow models that users can switch to.",
    "example": "curl http://127.0.0.1:5001/model/models",
    "method": "GET"
  },
  "/model/predict": {
    "description": "Classifies an image uploaded by the user as either 'damaged' or 'not damaged'.",
    "example": "curl -X POST -F 'image=@./data/damage/-93.795_30.03779.jpeg' http://localhost:5001/model/predict",
    "method": "POST"
  },
  "/model/summary": {
    "description": "Provides a textual summary of the currently loaded Tensor

## Route: /model/info

In [6]:
'''
Provides basic information about the currently loaded TensorFlow model.

GET: Returns a JSON object with the model's version, name, description,
total number of parameters, number of trainable parameters, and number of non-trainable parameters.

Example: curl http://127.0.0.1:5000/model/info
'''
url = f'http://127.0.0.1:{port}/model/info'
response = requests.get(url)
print(response.json())

{'description': 'Classify images of houses as damaged or not using the alt_lenet tensorflow model.', 'name': 'alt_lenet', 'non-trainable parameters:': '0', 'total parameters:': '2601153', 'trainable parameters:': '2601153', 'version': 'v1'}


## Route: /model/models

In [7]:
"""
Lists the available TensorFlow models that users can switch to.

GET: Returns a JSON object listing all available models and indicating the default model.

Example: curl http://127.0.0.1:5000/model/models
"""
url = f'http://127.0.0.1:{port}/model/models'
response = requests.get(url)
print(response.json())

{'available_models': ['vgg', 'resnet', 'xception', 'ann', 'lenet5', 'alt_lenet'], 'currently_loaded': 'alt_lenet', 'default_model': 'alt_lenet', 'note': 'alt_lenet is the default model loaded in and is the most accurate.'}


## Route: /model/summary

In [13]:
"""
Provides a textual summary of the currently loaded TensorFlow model's architecture.

GET: Returns a JSON array where each element is a string describing a layer in the model's architecture.

Example: curl http://127.0.0.1:5000/model/summary
"""
url = f'http://127.0.0.1:{port}/model/summary'
response = requests.get(url)
print(response.text)

[
  "Model: \"sequential_2\"",
  "_________________________________________________________________",
  " Layer (type)                Output Shape              Param #   ",
  "=================================================================",
  " conv2d_2 (Conv2D)           (None, 126, 126, 32)      896       ",
  "                                                                 ",
  " max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         ",
  " D)                                                              ",
  "                                                                 ",
  " conv2d_3 (Conv2D)           (None, 61, 61, 64)        18496     ",
  "                                                                 ",
  " max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         ",
  " g2D)                                                            ",
  "                                                                 ",
  " conv2d_4 (Conv2D)           (None, 28, 28,

## Route: /model/predict

In [31]:
'''
Classifies an image of a dog by the dog's breed.

POST: Expects a multipart/form-data request with an image file under the key "image".
Returns a JSON object with the classification result and a qualitative assessment of confidence.

Example: curl -X POST -F "image=@./data/damage/-93.795_30.03779.jpeg" http://localhost:5001/model/predict
'''
# damaged
#image_path = './data/images/affenpinscher-1.jpg'

# not damaged
image_path = './data/images/affenpinscher-1.jpg'

url = f'http://127.0.0.1:{port}/model/predict'

# Open the image in binary mode
with open(image_path, 'rb') as f:
    files = {'image': (image_path, f, 'image/jpeg')}
    response = requests.post(url, files=files)

print(response.text)

{
  "Outcome": "The model is confident that the building that is not damaged.",
  "result": [
    [
      1.5120656726708148e-08
    ]
  ]
}



## Route: /model/change

changing to lenet5 (from alt_lenet)

In [38]:
"""
Changes the TensorFlow model used by the server.

POST: Expects a JSON object with a key "model_name" that specifies the name of the new model to load.
Returns a JSON object with a message indicating successful model change and the path to the new model.

Example: curl -X POST -H 'Content-Type: application/json' -d '{"model_name": "inception"}' http://127.0.0.1:5000/model/change
"""

url = f'http://127.0.0.1:{port}/model/change'
data = {'model_name': 'inception'}
response = requests.post(url, json=data, headers={'Content-Type': 'application/json'})
print(response.text)

{
  "message": "Model changed to lenet5 successfully",
  "model_path": "models/lenet5.keras"
}



In [39]:
url = f'http://127.0.0.1:{port}/model/info'
response = requests.get(url)
print(response.json())

{'description': 'Classify images of houses as damaged or not using the lenet5 tensorflow model.', 'name': 'lenet5', 'non-trainable parameters:': '0', 'total parameters:': '1627961', 'trainable parameters:': '1627961', 'version': 'v1'}


In [41]:
image_path = './data/images/Airedale-1.jpg'

url = f'http://localhost:{port}/model/predict'

# Open the image in binary mode
with open(image_path, 'rb') as f:
    files = {'image': (image_path, f, 'image/jpeg')}
    response = requests.post(url, files=files)

print(response.text)

{
  "Outcome": "The model is confident that the building that is not damaged.",
  "result": [
    [
      3.212846655387125e-09
    ]
  ]
}



Note that above, the prediction is different from the first prediction in the /model/predict section with the alt-lenet 5 model. 


This verifies the model was changed. We can also change to Vgg, Resnet, and Ann models

### VGG16:

In [47]:
url = f'http://127.0.0.1:{port}/model/change'
data = {'model_name': 'vgg16'}
response = requests.post(url, json=data, headers={'Content-Type': 'application/json'})
print(response.json())

url = f'http://127.0.0.1:{port}/model/info'
response = requests.get(url)
print(response.json())

image_path = './data/images/Airedale-1.jpg'
url = f'http://localhost:{port}/model/predict'

# Open the image in binary mode
with open(image_path, 'rb') as f:
    files = {'image': (image_path, f, 'image/jpeg')}
    response = requests.post(url, files=files)

print(response.text)


{'message': 'Model changed to vgg successfully', 'model_path': 'models/vgg.keras'}
{'description': 'Classify images of houses as damaged or not using the vgg tensorflow model.', 'name': 'vgg', 'non-trainable parameters:': '14714688', 'total parameters:': '19077889', 'trainable parameters:': '4363201', 'version': 'v1'}
{
  "Outcome": "The model is confident that the building that is not damaged.",
  "result": [
    [
      0.00027580480673350394
    ]
  ]
}

